In [1]:
from pyscf import gto, scf, ao2mo, fci
from GeneralisedWick import *
from scipy.optimize import fsolve

In [2]:
def commutator(operator1, operator2):
    return operator1 * operator2 + (-1) * operator2 * operator1

In [3]:
def BCHSimilarityTransform(H, T, order):
    result = H
    for k in range(order):
        nestedCommutator = H
        for i in range(k + 1):
            nestedCommutator = commutator(nestedCommutator, T)
        result += (1 / factorial(k + 1)) * nestedCommutator
    return result

In [4]:
def projectionManifold(excitationLevel):
    return Tensor('\Phi', ['h'] * excitationLevel, ['p'] * excitationLevel)

In [5]:
fockTensor = Tensor("f", ['g'], ['g'])
h1Tensor = Tensor("h", ['g'], ['g'])
h2Tensor = Tensor("v", ['g', 'g'], ['g', 'g'])
t2Tensor = Tensor("{t_{2}}", ['p', 'p'], ['h', 'h'])

In [6]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
#mol.atom = 'Be 0 0 0'
#mol.basis = 'sto-3g'
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = '6-31g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

cisolver = fci.FCI(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1] * Nocc + [0] * (Norbs - Nocc)

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff, compact=False)

In [7]:
from pyscf import cc, ccn

In [167]:
trueCCD = cc.CCSD(mf)

In [121]:
trueCCD.update_amps

<bound method update_amps of <pyscf.cc.uccsd.UCCSD object at 0x7fc6fea58a90>>

In [168]:
old_update_amps = trueCCD.update_amps
def update_amps(t1, t2, eris):
    t1, t2 = old_update_amps(t1, t2, eris)
#    print(t1)
    return (np.zeros_like(t1[0]), np.zeros_like(t1[1])), t2
trueCCD.update_amps = update_amps

In [169]:
trueCCD.kernel()

(-0.027384725833890754,
 (array([[0., 0., 0.]]), array([[0., 0., 0.]])),
 (array([[[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]]]]),
  array([[[[-1.00150759e-01, -3.56674702e-17, -5.09840280e-02],
           [-1.91402868e-17, -4.56719620e-02, -6.00589164e-17],
           [-5.09840280e-02, -3.20448951e-17, -4.14451365e-02]]]]),
  array([[[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]]]])))

In [128]:
trueCCD.update_amps

<function __main__.update_amps(t1, t2, eris)>

In [143]:
trueCCD.

SyntaxError: invalid syntax (<ipython-input-143-54f6fde26642>, line 1)

In [10]:
trueAmplitudeArray = trueCCD[2][1].swapaxes(0,2).swapaxes(1,3)

In [11]:
trueAmplitudeArray

array([[[[-1.00108543e-01]],

        [[-8.89555065e-17]],

        [[-5.16134266e-02]]],


       [[[-6.87692457e-17]],

        [[-4.53856765e-02]],

        [[-5.19382925e-17]]],


       [[[-5.16134266e-02]],

        [[-2.31946442e-17]],

        [[-4.22705615e-02]]]])

In [12]:
#t2Tensor.array

In [13]:
def get2bodyHamiltonianArray(mf_):
    eri = ao2mo.kernel(mf_.mol, mf_.mo_coeff)
    Norbs_ = mf_.mol.nao
    hamiltonian2BodyArray = np.zeros((Norbs_,Norbs_,Norbs_,Norbs_))
    for p in range(Norbs_):
        for q in range(Norbs_):
            for r in range(p + 1):
                for s in range(q + 1):
                    x = int(p + Norbs_ * r - 0.5 * r * (r + 1))
                    y = int(q + Norbs_ * s - 0.5 * s * (s + 1))
                    if p == r and q == s:
                        hamiltonian2BodyArray[p,q,r,s] += eri[x, y]
                    else:
                        hamiltonian2BodyArray[p,q,r,s] += eri[x, y]
                        hamiltonian2BodyArray[r,s,p,q] += np.conjugate(eri[x, y])
    return hamiltonian2BodyArray

In [14]:
h1Tensor.array = h1
h2Tensor.array = eri.reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)
t2Tensor.getShape(vacuum)

In [15]:
#t2Tensor.array[0, 0, 0, 0] = -0.275333314977893 * 0.5
t2Tensor.array = np.zeros(t2Tensor.array.shape)
#t2Tensor.array = trueAmplitudeArray

In [16]:
h1

array([[-1.17699652e+00,  4.86344149e-16, -1.53193765e-01,
         4.98835655e-18],
       [ 7.21155607e-16, -5.73866278e-01,  1.87640153e-16,
        -2.03002602e-01],
       [-1.53193765e-01, -1.08784094e-16, -1.08071573e-01,
        -8.91361467e-17],
       [-3.81087394e-17, -2.03002602e-01, -1.51037500e-16,
         1.63887344e-01]])

In [17]:
mf.mo_energy

NPArrayWithTag([-0.56467538,  0.20875728,  0.82891329,  1.28023505])

In [18]:
eri

array([[ 6.12321260e-01, -3.34968904e-16,  1.53193685e-01,
        -2.79148766e-16, -3.09636953e-16,  4.37186032e-01,
        -4.29537612e-17,  1.41953596e-01,  1.53193685e-01,
        -5.32258994e-17,  5.20155049e-01,  9.18276830e-17,
        -3.46655211e-16,  1.41953596e-01,  9.60641438e-17,
         6.21237009e-01],
       [-3.91069650e-16,  9.17484084e-02,  1.32010509e-17,
         8.09045221e-02,  9.17484084e-02, -3.69309873e-16,
        -8.80797318e-03,  3.59280805e-17,  1.10514385e-17,
        -8.80797318e-03, -6.91883539e-18,  9.87001421e-02,
         8.09045221e-02,  5.39168144e-17,  9.87001421e-02,
        -9.82161813e-16],
       [ 1.53193685e-01,  5.40844729e-18,  1.03325146e-01,
         8.10671166e-17, -8.44973838e-19,  5.78044849e-02,
        -2.74581992e-17,  7.76385037e-02,  1.03325146e-01,
        -2.29972719e-17,  1.19739160e-01,  3.32295076e-16,
         7.65543083e-17,  7.76385037e-02,  3.18137536e-16,
         1.88932088e-01],
       [-2.31371301e-16,  8.09045221e

In [19]:
h2Tensor.array

array([[[[ 6.12321260e-01, -3.09636953e-16,  1.53193685e-01,
          -3.46655211e-16],
         [-3.91069650e-16,  9.17484084e-02,  1.10514385e-17,
           8.09045221e-02],
         [ 1.53193685e-01, -8.44973838e-19,  1.03325146e-01,
           7.65543083e-17],
         [-2.31371301e-16,  8.09045221e-02,  6.29742429e-17,
           1.26126117e-01]],

        [[-3.34968904e-16,  4.37186032e-01, -5.32258994e-17,
           1.41953596e-01],
         [ 9.17484084e-02, -3.69309873e-16, -8.80797318e-03,
           5.39168144e-17],
         [ 5.40844729e-18,  5.78044849e-02, -2.29972719e-17,
           7.76385037e-02],
         [ 8.09045221e-02, -1.27537884e-16,  3.35316491e-02,
          -4.40679193e-16]],

        [[ 1.53193685e-01, -4.29537612e-17,  5.20155049e-01,
           9.60641438e-17],
         [ 1.32010509e-17, -8.80797318e-03, -6.91883539e-18,
           9.87001421e-02],
         [ 1.03325146e-01, -2.74581992e-17,  1.19739160e-01,
           3.18137536e-16],
         [ 5.4629

In [20]:
#fock = mf.get_fock(h1e=h1, vhf=eri)

In [21]:
#print(fock)
#print(np.linalg.eig(fock))

In [22]:
eri.reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)

array([[[[ 6.12321260e-01, -3.09636953e-16,  1.53193685e-01,
          -3.46655211e-16],
         [-3.91069650e-16,  9.17484084e-02,  1.10514385e-17,
           8.09045221e-02],
         [ 1.53193685e-01, -8.44973838e-19,  1.03325146e-01,
           7.65543083e-17],
         [-2.31371301e-16,  8.09045221e-02,  6.29742429e-17,
           1.26126117e-01]],

        [[-3.34968904e-16,  4.37186032e-01, -5.32258994e-17,
           1.41953596e-01],
         [ 9.17484084e-02, -3.69309873e-16, -8.80797318e-03,
           5.39168144e-17],
         [ 5.40844729e-18,  5.78044849e-02, -2.29972719e-17,
           7.76385037e-02],
         [ 8.09045221e-02, -1.27537884e-16,  3.35316491e-02,
          -4.40679193e-16]],

        [[ 1.53193685e-01, -4.29537612e-17,  5.20155049e-01,
           9.60641438e-17],
         [ 1.32010509e-17, -8.80797318e-03, -6.91883539e-18,
           9.87001421e-02],
         [ 1.03325146e-01, -2.74581992e-17,  1.19739160e-01,
           3.18137536e-16],
         [ 5.4629

In [23]:
fock = h1
for p in range(Norbs):
    for q in range(Norbs):
        fock[p,q] += sum([2 * h2Tensor.array[p,i,q,i] - h2Tensor.array[p,i,i,q] for i in range(Nocc)])

In [24]:
fock

array([[-5.64675261e-01,  1.38418012e-17, -8.01293097e-08,
        -1.11099034e-16],
       [ 3.81170846e-16,  2.08757379e-01,  3.85998797e-17,
         6.79839383e-08],
       [-8.01293098e-08, -1.82623702e-16,  8.28913379e-01,
         1.69114098e-16],
       [-2.71854981e-16,  6.79839386e-08,  7.40103067e-17,
         1.28023524e+00]])

In [25]:
fockTensor.array = fock

In [26]:
np.linalg.eig(fock)

(array([-0.56467526,  0.82891338,  0.20875738,  1.28023524]),
 array([[-1.00000000e+00,  5.74985382e-08,  1.78965590e-17,
         -6.02192152e-17],
        [ 4.92830066e-16,  1.30536998e-17,  1.00000000e+00,
          6.34487566e-08],
        [-5.74985383e-08, -1.00000000e+00,  2.78319954e-16,
          3.81142508e-16],
        [-1.47354037e-16,  1.63985677e-16, -6.34487569e-08,
          1.00000000e+00]]))

In [27]:
mf.mo_energy

NPArrayWithTag([-0.56467538,  0.20875728,  0.82891329,  1.28023505])

Hard code spin-free equations from Bartlett & Shavitt Chapter 12

In [28]:
fockDiagrams = fockTensor.getAllDiagrams(vacuum)
h2Diagrams = h2Tensor.getAllDiagrams(vacuum)

In [29]:
print(*fockDiagrams)

f_{h}^{h} f_{h}^{p} f_{p}^{h} f_{p}^{p}


In [30]:
print([fockDiagram.array.shape for fockDiagram in fockDiagrams])

[(1, 1), (1, 3), (3, 1), (3, 3)]


In [31]:
print(*h2Diagrams)

v_{hh}^{hh} v_{hh}^{hp} v_{hh}^{ph} v_{hh}^{pp} v_{hp}^{hh} v_{hp}^{hp} v_{hp}^{ph} v_{hp}^{pp} v_{ph}^{hh} v_{ph}^{hp} v_{ph}^{ph} v_{ph}^{pp} v_{pp}^{hh} v_{pp}^{hp} v_{pp}^{ph} v_{pp}^{pp}


In [32]:
print([h2Diagram.array.shape for h2Diagram in h2Diagrams])

[(1, 1, 1, 1), (1, 1, 1, 3), (1, 1, 3, 1), (1, 1, 3, 3), (1, 3, 1, 1), (1, 3, 1, 3), (1, 3, 3, 1), (1, 3, 3, 3), (3, 1, 1, 1), (3, 1, 1, 3), (3, 1, 3, 1), (3, 1, 3, 3), (3, 3, 1, 1), (3, 3, 1, 3), (3, 3, 3, 1), (3, 3, 3, 3)]


In [33]:
I = Index("I", True)
J = Index("J", True)
K = Index("K", True)
L = Index("L", True)
A = Index("A", False)
B = Index("B", False)
C = Index("C", False)
D = Index("D", False)

In [34]:
D1 = TensorSum([TensorProduct([h2Diagrams[12]], vertexList=[Vertex(h2Diagrams[12], [A,B], [I,J])])])

In [35]:
D2a = TensorSum([TensorProduct([fockDiagrams[3], t2Tensor], vertexList=[Vertex(fockDiagrams[3], [B], [C]), Vertex(t2Tensor, [A,C], [I,J])]), TensorProduct([fockDiagrams[3], t2Tensor], vertexList=[Vertex(fockDiagrams[3], [A], [C]), Vertex(t2Tensor, [B,C], [J,I])])])

In [36]:
D2b = TensorSum([TensorProduct([fockDiagrams[0], t2Tensor], vertexList=[Vertex(fockDiagrams[0], [K], [J]), Vertex(t2Tensor, [A,B], [I,K])]), TensorProduct([fockDiagrams[0], t2Tensor], vertexList=[Vertex(fockDiagrams[0], [K], [I]), Vertex(t2Tensor, [B,A], [J,K])])])

In [37]:
D2c = TensorSum([TensorProduct([h2Diagrams[15], t2Tensor], vertexList=[Vertex(h2Diagrams[15], [A,B], [C,D]), Vertex(t2Tensor, [C,D], [I,J])]), TensorProduct([h2Diagrams[15], t2Tensor], vertexList=[Vertex(h2Diagrams[15], [B,A], [C,D]), Vertex(t2Tensor, [C,D], [J,I])])])

In [38]:
D2d = TensorSum([TensorProduct([h2Diagrams[0], t2Tensor], vertexList=[Vertex(h2Diagrams[0], [K,L], [I,J]), Vertex(t2Tensor, [A,B], [K,L])]), TensorProduct([h2Diagrams[0], t2Tensor], vertexList=[Vertex(h2Diagrams[0], [K,L], [J,I]), Vertex(t2Tensor, [B,A], [K,L])])])

In [39]:
D2e = TensorSum([TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,B], [C,J]), Vertex(t2Tensor, [A,C], [I,K])]), TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,A], [C,I]), Vertex(t2Tensor, [B,C], [J,K])])])

In [40]:
D2ex1 = TensorSum([TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,B], [C,J]), Vertex(t2Tensor, [A,C], [K,I])]), TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,A], [C,I]), Vertex(t2Tensor, [B,C], [K,J])])])

In [41]:
D2ex2 = TensorSum([TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,B], [I,C]), Vertex(t2Tensor, [A,C], [K,J])]), TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,A], [J,C]), Vertex(t2Tensor, [B,C], [K,I])])])

In [42]:
D2ex3 = TensorSum([TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,B], [J,C]), Vertex(t2Tensor, [A,C], [I,K])]), TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,A], [I,C]), Vertex(t2Tensor, [B,C], [J,K])])])

In [43]:
D3a = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [I,J]), Vertex(t2Tensor, [A,B], [K,L])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [J,I]), Vertex(t2Tensor, [B,A], [K,L])])])

In [44]:
D3b = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,C], [I,K]), Vertex(t2Tensor, [D,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,C], [J,K]), Vertex(t2Tensor, [D,A], [L,I])])])

In [45]:
D3bx1 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,C], [I,K]), Vertex(t2Tensor, [D,B], [J,L])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,C], [J,K]), Vertex(t2Tensor, [D,A], [I,L])])])

In [46]:
D3bx2 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,A], [I,K]), Vertex(t2Tensor, [B,D], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,B], [J,K]), Vertex(t2Tensor, [A,D], [L,I])])])

In [47]:
D3bx3 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,D], [I,K]), Vertex(t2Tensor, [C,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,D], [J,K]), Vertex(t2Tensor, [C,A], [L,I])])])

In [48]:
D3bx4 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,D], [K,I]), Vertex(t2Tensor, [C,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,D], [K,J]), Vertex(t2Tensor, [C,A], [L,I])])])

In [49]:
D3bx5 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,B], [I,L]), Vertex(t2Tensor, [A,D], [K,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,A], [J,L]), Vertex(t2Tensor, [B,D], [K,I])])])

In [50]:
D3c = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [K,I]), Vertex(t2Tensor, [A,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [K,J]), Vertex(t2Tensor, [B,A], [L,I])])])

In [51]:
D3cx = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [I,K]), Vertex(t2Tensor, [A,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [J,K]), Vertex(t2Tensor, [B,A], [L,I])])])

In [52]:
D3d = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,A], [K,L]), Vertex(t2Tensor, [D,B], [I,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,B], [K,L]), Vertex(t2Tensor, [D,A], [J,I])])])

In [53]:
D3dx = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,C], [K,L]), Vertex(t2Tensor, [D,B], [I,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,C], [K,L]), Vertex(t2Tensor, [D,A], [J,I])])])

In [54]:
amplitudeDiagrams = [D1, D2a, D2b, D2c, D2d, D2e, D2ex1, D2ex2, D2ex3, D3a, D3b, D3bx1, D3bx2, D3bx3, D3bx4, D3bx5, D3c, D3cx, D3d, D3dx]
amplitudeFactors = [1., 1., -1., 0.5, 0.5, 2., -1., -1., -1., 0.5, 2., -2., 0.5, -1., 1., 0.5, -2., 1., -2., 1.]

In [55]:
print(D2a)

1.0 * f_{B}^{C}{t_{2}}_{AC}^{IJ}
 + 1.0 * f_{A}^{C}{t_{2}}_{BC}^{JI}


In [56]:
D2a.summandList[0].vertexList[1].tensor.array.shape

(3, 3, 1, 1)

In [57]:
contractTensorSum(TensorSum([D2a.summandList[0]])).shape

(3, 3, 1, 1)

In [58]:
def testContractedArray(tensorProduct_, targetLowerIndexList=[A,B], targetUpperIndexList=[I,J]):
    lowerIndexList = list(itertools.chain.from_iterable([vertex.lowerIndices for vertex in tensorProduct_.vertexList]))
    upperIndexList = list(itertools.chain.from_iterable([vertex.upperIndices for vertex in tensorProduct_.vertexList]))
    lowerIndexLetters = string.ascii_lowercase[:len(lowerIndexList)]
    upperIndexLetters = ''
    freeLowerIndexMask = np.ones(len(lowerIndexList))
    freeUpperIndexMask = np.ones(len(upperIndexList))
    nFreeUpperIndices = 0
    for uI, upperIndex in enumerate(upperIndexList):
        free = True
        for lI, lowerIndex in enumerate(lowerIndexList):
            if upperIndex == lowerIndex:
                #print(upperIndex, lowerIndex)
                upperIndexLetters += lowerIndexLetters[lI]
                freeLowerIndexMask[lI] = 0
                freeUpperIndexMask[uI] = 0
                free = False
        if free:
            upperIndexLetters += string.ascii_lowercase[len(lowerIndexList) + nFreeUpperIndices]
            nFreeUpperIndices += 1
    newLowerIndexList = [lowerIndex for lI, lowerIndex in enumerate(lowerIndexList) if freeLowerIndexMask[lI]]
    newUpperIndexList = [upperIndex for uI, upperIndex in enumerate(upperIndexList) if freeUpperIndexMask[uI]]
    summandZero = False
    if targetLowerIndexList == None and targetUpperIndexList == None:
        targetLowerIndexList = newLowerIndexList
        targetUpperIndexList = newUpperIndexList
        summandZero = True
    freeLowerIndexLetters = "".join([lowerIndexLetters[lowerIndexList.index(lowerIndex)] for lowerIndex in targetLowerIndexList])
    freeUpperIndexLetters = "".join([upperIndexLetters[upperIndexList.index(upperIndex)] for upperIndex in targetUpperIndexList])
    einsumSubstrings = []
    start = 0
    for vertex in tensorProduct_.vertexList:
        end = start + vertex.excitationRank
        einsumSubstring = lowerIndexLetters[start:end] + upperIndexLetters[start:end]
        einsumSubstrings.append(einsumSubstring)
        start = end
    einsumString = ",".join(einsumSubstrings)
    einsumString += '->' + freeLowerIndexLetters + freeUpperIndexLetters
    print(*lowerIndexList, *upperIndexList)
    print(*lowerIndexLetters, *upperIndexLetters)
    print(*newLowerIndexList, *newUpperIndexList)
    print(*freeLowerIndexLetters, *freeUpperIndexLetters)
    print(einsumString)
    contribution = tensorProduct_.prefactor * np.einsum(einsumString, *[vertex.tensor.array for vertex in tensorProduct_.vertexList])
    #print(*newLowerIndexList, *newUpperIndexList)
    #if summandZero:
    #    return contribution, newLowerIndexList, newUpperIndexList
    return contribution

In [59]:
t2Tensor.array.shape

(3, 3, 1, 1)

In [60]:
tCA = testContractedArray(D2a.summandList[0])
print(D2a.summandList[1])
print(tCA)
print(tCA.shape)
#print(*tCA[1])
#print(*tCA[2])

B A C C I J
a b c c d e
B A I J
b a d e
ac,bcde->bade
1.0 * f_{A}^{C}{t_{2}}_{BC}^{JI}
[[[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]]
(3, 3, 1, 1)


In [61]:
amplitudes = TensorSum([])
for d, diagram in enumerate(amplitudeDiagrams):
    for summand in diagram.summandList:
        amplitudes.summandList.append(copy(summand))
        amplitudes.summandList[-1].prefactor *= amplitudeFactors[d]
    print(diagram)
    print(contractTensorSum(diagram))
#    print(amplitudeFactors[d] * contractTensorSum(diagram))

1.0 * v_{AB}^{IJ}
[[[[9.17484084e-02]]

  [[6.38930743e-17]]

  [[8.09045221e-02]]]


 [[[4.62971229e-17]]

  [[1.03325146e-01]]

  [[7.33666138e-17]]]


 [[[8.09045221e-02]]

  [[2.76838403e-17]]

  [[1.26126117e-01]]]]
1.0 * f_{B}^{C}{t_{2}}_{AC}^{IJ}
 + 1.0 * f_{A}^{C}{t_{2}}_{BC}^{JI}
[[[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]]
1.0 * f_{K}^{J}{t_{2}}_{AB}^{IK}
 + 1.0 * f_{K}^{I}{t_{2}}_{BA}^{JK}
[[[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]]
1.0 * v_{AB}^{CD}{t_{2}}_{CD}^{IJ}
 + 1.0 * v_{BA}^{CD}{t_{2}}_{CD}^{JI}
[[[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]]
1.0 * v_{KL}^{IJ}{t_{2}}_{AB}^{KL}
 + 1.0 * v_{KL}^{JI}{t_{2}}_{BA}^{KL}
[[[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]]
1.0 * v_{KB}^{CJ}{t_{2}}_{AC}^{IK}
 + 1.0 * v_{KA}^{CI}{t_{2}}_{BC}^{JK}
[[[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]



In [62]:
#t2Tensor.array[0, 0, 0, 0] = -0.38

In [63]:
h2Tensor.array = get2bodyHamiltonianArray(mf)

In [64]:
residualTerms = [amplitudeFactors[d] * contractTensorSum(diagram) for d, diagram in enumerate(amplitudeDiagrams)]

In [65]:
sum([testContractedArray(term) for term in amplitudes.summandList])

A B I J
a b c d
A B I J
a b c d
abcd->abcd
B A C C I J
a b c c d e
B A I J
b a d e
ac,bcde->bade
A B C C J I
a b c c d e
A B J I
a b e d
ac,bcde->abed
K A B J I K
a b c d e a
A B J I
b c e d
ad,bcea->bced
K B A I J K
a b c d e a
B A I J
c b d e
ad,bcea->cbde
A B C D C D I J
a b c d c d e f
A B I J
a b e f
abcd,cdef->abef
B A C D C D J I
a b c d c d e f
B A J I
b a f e
abcd,cdef->bafe
K L A B I J K L
a b c d e f a b
A B I J
c d e f
abef,cdab->cdef
K L B A J I K L
a b c d e f a b
B A J I
d c f e
abef,cdab->dcfe
K B A C C J I K
a b c d d e f a
B A J I
c b f e
abde,cdfa->cbfe
K A B C C I J K
a b c d d e f a
A B I J
b c e f
abde,cdfa->bcef
K B A C C J K I
a b c d d e a f
B A J I
c b f e
abde,cdaf->cbfe
K A B C C I K J
a b c d d e a f
A B I J
b c e f
abde,cdaf->bcef
K B A C I C K J
a b c d e d a f
B A I J
c b e f
abed,cdaf->cbef
K A B C J C K I
a b c d e d a f
A B J I
b c f e
abed,cdaf->bcfe
K B A C J C I K
a b c d e d f a
B A J I
c b f e
abed,cdfa->cbfe
K A B C I C J K
a b c d e d f a
A B I

array([[[[9.17484084e-02]],

        [[6.38930743e-17]],

        [[8.09045221e-02]]],


       [[[4.62971229e-17]],

        [[1.03325146e-01]],

        [[7.33666138e-17]]],


       [[[8.09045221e-02]],

        [[2.76838403e-17]],

        [[1.26126117e-01]]]])

In [66]:
contractTensorSum(amplitudes)

array([[[[9.17484084e-02]],

        [[6.38930743e-17]],

        [[8.09045221e-02]]],


       [[[4.62971229e-17]],

        [[1.03325146e-01]],

        [[7.33666138e-17]]],


       [[[8.09045221e-02]],

        [[2.76838403e-17]],

        [[1.26126117e-01]]]])

In [67]:
residualTerms

[array([[[[9.17484084e-02]],
 
         [[6.38930743e-17]],
 
         [[8.09045221e-02]]],
 
 
        [[[4.62971229e-17]],
 
         [[1.03325146e-01]],
 
         [[7.33666138e-17]]],
 
 
        [[[8.09045221e-02]],
 
         [[2.76838403e-17]],
 
         [[1.26126117e-01]]]]),
 array([[[[0.]],
 
         [[0.]],
 
         [[0.]]],
 
 
        [[[0.]],
 
         [[0.]],
 
         [[0.]]],
 
 
        [[[0.]],
 
         [[0.]],
 
         [[0.]]]]),
 array([[[[-0.]],
 
         [[-0.]],
 
         [[-0.]]],
 
 
        [[[-0.]],
 
         [[-0.]],
 
         [[-0.]]],
 
 
        [[[-0.]],
 
         [[-0.]],
 
         [[-0.]]]]),
 array([[[[0.]],
 
         [[0.]],
 
         [[0.]]],
 
 
        [[[0.]],
 
         [[0.]],
 
         [[0.]]],
 
 
        [[[0.]],
 
         [[0.]],
 
         [[0.]]]]),
 array([[[[0.]],
 
         [[0.]],
 
         [[0.]]],
 
 
        [[[0.]],
 
         [[0.]],
 
         [[0.]]],
 
 
        [[[0.]],
 
         [[0.]],
 
         [[0.

In [68]:
constant = sum(residualTerms[:1])
linearFock = sum(residualTerms[1:3])
linearFluctuation = sum(residualTerms[3:9])
quadratic = sum(residualTerms[9:])

In [69]:
supposedLinearFluctuation = residualTerms[3] - residualTerms[4]
print(supposedLinearFluctuation)

[[[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]]


In [70]:
print(constant)
print(linearFock)
print(linearFluctuation)
print(quadratic)

[[[[9.17484084e-02]]

  [[6.38930743e-17]]

  [[8.09045221e-02]]]


 [[[4.62971229e-17]]

  [[1.03325146e-01]]

  [[7.33666138e-17]]]


 [[[8.09045221e-02]]

  [[2.76838403e-17]]

  [[1.26126117e-01]]]]
[[[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]]
[[[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]]
[[[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]]


In [71]:
constant + linearFock + supposedLinearFluctuation + quadratic

array([[[[9.17484084e-02]],

        [[6.38930743e-17]],

        [[8.09045221e-02]]],


       [[[4.62971229e-17]],

        [[1.03325146e-01]],

        [[7.33666138e-17]]],


       [[[8.09045221e-02]],

        [[2.76838403e-17]],

        [[1.26126117e-01]]]])

In [72]:
sum(residualTerms)

array([[[[9.17484084e-02]],

        [[6.38930743e-17]],

        [[8.09045221e-02]]],


       [[[4.62971229e-17]],

        [[1.03325146e-01]],

        [[7.33666138e-17]]],


       [[[8.09045221e-02]],

        [[2.76838403e-17]],

        [[1.26126117e-01]]]])

In [73]:
sum(residualTerms[9:])

array([[[[0.]],

        [[0.]],

        [[0.]]],


       [[[0.]],

        [[0.]],

        [[0.]]],


       [[[0.]],

        [[0.]],

        [[0.]]]])

In [74]:
sum(residualTerms[:5]) + sum(residualTerms[9:])

array([[[[9.17484084e-02]],

        [[6.38930743e-17]],

        [[8.09045221e-02]]],


       [[[4.62971229e-17]],

        [[1.03325146e-01]],

        [[7.33666138e-17]]],


       [[[8.09045221e-02]],

        [[2.76838403e-17]],

        [[1.26126117e-01]]]])

In [75]:
Ej = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [A,B]), Vertex(t2Tensor, [A,B], [I,J])])])
Ek = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [B,A]), Vertex(t2Tensor, [A,B], [I,J])])])

In [76]:
ECCD = 2 * contractTensorSum(Ej) - contractTensorSum(Ek)
print(ECCD)

0.0


In [77]:
def getCCDEnergy(doublesTensor):
    Ej = TensorSum([TensorProduct([h2Diagrams[3], doublesTensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [A,B]), Vertex(doublesTensor, [A,B], [I,J])])])
    Ek = TensorSum([TensorProduct([h2Diagrams[3], doublesTensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [B,A]), Vertex(doublesTensor, [A,B], [I,J])])])
    return 2 * contractTensorSum(Ej) - contractTensorSum(Ek)

In [78]:
t2Tensor.array * ECCD

array([[[[0.]],

        [[0.]],

        [[0.]]],


       [[[0.]],

        [[0.]],

        [[0.]]],


       [[[0.]],

        [[0.]],

        [[0.]]]])

In [79]:
print(Ej)
print(Ek)

1.0 * v_{IJ}^{AB}{t_{2}}_{AB}^{IJ}
1.0 * v_{IJ}^{BA}{t_{2}}_{AB}^{IJ}


Coupled cluster

In [80]:
print(t2Tensor.array.reshape((-1)))

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [81]:
residualTensor = Tensor("R", ['p', 'p'], ['h', 'h'])

In [82]:
residualTensor.array = contractTensorSum(amplitudes)

In [83]:
print(residualTensor.array.reshape((-1)))

[9.17484084e-02 6.38930743e-17 8.09045221e-02 4.62971229e-17
 1.03325146e-01 7.33666138e-17 8.09045221e-02 2.76838403e-17
 1.26126117e-01]


In [84]:
print(t2Tensor.array)

[[[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]]


In [85]:
def doublesAmplitudesToVector(doublesAmplitudeTensor):
    vec = []
#    for i in range()
    return vec

In [86]:
def testAmplitudes(amplitudeVector):
    t2Tensor.array = amplitudeVector.reshape(t2Tensor.array.shape)
    residualTensor.array = contractTensorSum(amplitudes)
    return residualTensor.array.reshape((-1))

In [87]:
def getDoublesMP2Amplitudes(doublesTensor):
    mp2Amplitudes = np.zeros_like(doublesTensor.array)
    for i in range(mp2Amplitudes.shape[0]):
        for j in range(mp2Amplitudes.shape[1]):
            for k in range(mp2Amplitudes.shape[2]):
                for l in range(mp2Amplitudes.shape[3]):
#                    mp2Amplitudes[i,j,k,l] = -(2 - (i==j)) * h2Diagrams[12].array[i,j,k,l] / (fock[i + Nocc, i + Nocc] + fock[j + Nocc, j + Nocc] - fock[k, k] - fock[l, l])
                    mp2Amplitudes[i,j,k,l] = -h2Diagrams[12].array[i,j,k,l] / (fock[i + Nocc, i + Nocc] + fock[j + Nocc, j + Nocc] - fock[k, k] - fock[l, l])
    return mp2Amplitudes

In [88]:
def iterateDoublesAmplitudes(doublesTensor, residual):
    amplitudes = doublesTensor.array
    for i in range(amplitudes.shape[0]):
        for j in range(amplitudes.shape[1]):
            for k in range(amplitudes.shape[2]):
                for l in range(amplitudes.shape[3]):
#                    mp2Amplitudes[i,j,k,l] = (2 - (i==j)) * residual.array[i,j,k,l] / (fock[i + Nocc, i + Nocc] + fock[j + Nocc, j + Nocc] - fock[k, k] - fock[l, l])
                    amplitudes[i,j,k,l] -= residual.array[i,j,k,l] / (fock[i + Nocc, i + Nocc] + fock[j + Nocc, j + Nocc] - fock[k, k] - fock[l, l])
    return amplitudes

In [89]:
iterateDoublesAmplitudes(t2Tensor, h2Diagrams[12])

array([[[[-5.93124751e-02]],

        [[-2.94842856e-17]],

        [[-3.08991288e-02]]],


       [[[-2.13644062e-17]],

        [[-3.70716089e-02]],

        [[-2.26545108e-17]]],


       [[[-3.08991288e-02]],

        [[-8.54835499e-18]],

        [[-3.41821777e-02]]]])

In [90]:
t2Tensor.array = np.zeros_like(t2Tensor.array)

In [91]:
Energy = getCCDEnergy(t2Tensor)

In [92]:
print(t2Tensor.array)

[[[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]


 [[[0.]]

  [[0.]]

  [[0.]]]]


In [93]:
print(getDoublesMP2Amplitudes(t2Tensor))

[[[[-5.93124751e-02]]

  [[-2.94842856e-17]]

  [[-3.08991288e-02]]]


 [[[-2.13644062e-17]]

  [[-3.70716089e-02]]

  [[-2.26545108e-17]]]


 [[[-3.08991288e-02]]

  [[-8.54835499e-18]]

  [[-3.41821777e-02]]]]


In [94]:
def convergeDoublesAmplitudes():
    t2Tensor.array = np.zeros_like(t2Tensor.array)
#    t2Tensor.array = trueAmplitudeArray
    Energy = getCCDEnergy(t2Tensor)
    residualTensor.array = h2Diagrams[12].array
    while True:
        print(Energy)
        t2Tensor.array = iterateDoublesAmplitudes(t2Tensor, residualTensor)
        residualTensor.array = contractTensorSum(amplitudes)
        Energy = getCCDEnergy(t2Tensor)
#        newEnergy = getCCDEnergy(t2Tensor)
#        if abs(newEnergy - Energy) < 0.00000001:
        if np.all(abs(residualTensor.array) < 0.0000000000000001):
            break
#        Energy = newEnergy
    print(Energy)
    print(t2Tensor.array)

In [95]:
residualTensor.array

array([[[[9.17484084e-02]],

        [[6.38930743e-17]],

        [[8.09045221e-02]]],


       [[[4.62971229e-17]],

        [[1.03325146e-01]],

        [[7.33666138e-17]]],


       [[[8.09045221e-02]],

        [[2.76838403e-17]],

        [[1.26126117e-01]]]])

In [170]:
trueCCD.kernel()

(-0.027384725833890754,
 (array([[0., 0., 0.]]), array([[0., 0., 0.]])),
 (array([[[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]]]]),
  array([[[[-1.00150759e-01, -3.56674702e-17, -5.09840280e-02],
           [-1.91402868e-17, -4.56719620e-02, -6.00589164e-17],
           [-5.09840280e-02, -3.20448951e-17, -4.14451365e-02]]]]),
  array([[[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]]]])))

In [171]:
convergeDoublesAmplitudes()

0.0
-0.01858327844380032
-0.024338688080780043
-0.026258028934063937
-0.026948474547471984
-0.02721103919349661
-0.02731445883556027
-0.027356042437742843
-0.02737295871979066
-0.027379885135130934
-0.027382731413169607
-0.027383903382645118
-0.027384386487644026
-0.027384585756915897
-0.027384667980100502
-0.02738470191425738
-0.027384715920808215
-0.02738472170250636
-0.027384724089204397
-0.02738472507446315
-0.027384725481196456
-0.027384725649105195
-0.027384725718422157
-0.027384725747038055
-0.0273847257588515
-0.02738472576372844
-0.02738472576574178
-0.02738472576657294
-0.027384725766916064
-0.027384725767057715
-0.027384725767116203
-0.02738472576714034
-0.027384725767150304
-0.027384725767154425
-0.027384725767156125
-0.027384725767156816
-0.0273847257671571
-0.027384725767157225
-0.02738472576715727
-0.027384725767157295
[[[[-1.00150758e-01]]

  [[-1.92328091e-16]]

  [[-5.09840279e-02]]]


 [[[ 1.24347205e-16]]

  [[-4.56719620e-02]]

  [[-2.53910966e-16]]]


 [[[-5.09840

In [98]:
residualTensor.array

array([[[[ 4.16333634e-17]],

        [[ 8.40009911e-18]],

        [[ 9.90960786e-17]]],


       [[[-9.19585231e-18]],

        [[-3.51823605e-17]],

        [[ 1.92923914e-17]]],


       [[[ 4.01154804e-17]],

        [[-2.63903821e-17]],

        [[-3.64291930e-17]]]])

In [99]:
trueCCD

(-0.02754924051347242,
 (array([[-5.46837800e-16,  8.78987007e-03,  1.66942383e-16]]),
  array([[-4.90379607e-16,  8.78987007e-03,  1.71047985e-16]])),
 (array([[[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]]]]),
  array([[[[-1.00108543e-01, -8.89555065e-17, -5.16134266e-02],
           [-6.87692457e-17, -4.53856765e-02, -5.19382925e-17],
           [-5.16134266e-02, -2.31946442e-17, -4.22705615e-02]]]]),
  array([[[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]]]])))

In [100]:
t2Tensor.array

array([[[[-1.00150758e-01]],

        [[-1.92328091e-16]],

        [[-5.09840279e-02]]],


       [[[ 1.24347205e-16]],

        [[-4.56719620e-02]],

        [[-2.53910966e-16]]],


       [[[-5.09840279e-02]],

        [[ 2.96229110e-16]],

        [[-4.14451365e-02]]]])

In [101]:
trueAmplitudeArray

array([[[[-1.00108543e-01]],

        [[-8.89555065e-17]],

        [[-5.16134266e-02]]],


       [[[-6.87692457e-17]],

        [[-4.53856765e-02]],

        [[-5.19382925e-17]]],


       [[[-5.16134266e-02]],

        [[-2.31946442e-17]],

        [[-4.22705615e-02]]]])

In [102]:
t2Tensor.array = getDoublesMP2Amplitudes(t2Tensor)
fsolve(testAmplitudes, t2Tensor.array.reshape((-1)))

array([-1.25193285e-01,  1.21574507e-09, -9.59743061e-02, -2.11597919e-09,
       -4.67115764e-02,  3.88925452e-10,  4.50300633e-02, -8.79183171e-10,
       -2.76078403e-02])

In [103]:
t2Tensor.array

array([[[[-1.25193285e-01]],

        [[ 8.09790572e-10]],

        [[-9.59743043e-02]]],


       [[[-1.40887435e-09]],

        [[-4.67115763e-02]],

        [[ 2.63123857e-10]]],


       [[[ 4.50300594e-02]],

        [[-5.88652350e-10]],

        [[-2.76078409e-02]]]])

In [104]:
2 * contractTensorSum(Ej) - contractTensorSum(Ek)

-0.023916454602257702

In [105]:
trueAmplitudeArray

array([[[[-1.00108543e-01]],

        [[-8.89555065e-17]],

        [[-5.16134266e-02]]],


       [[[-6.87692457e-17]],

        [[-4.53856765e-02]],

        [[-5.19382925e-17]]],


       [[[-5.16134266e-02]],

        [[-2.31946442e-17]],

        [[-4.22705615e-02]]]])

In [106]:
residualTensor.array

array([[[[ 7.76592766e-13]],

        [[ 1.76101135e-13]],

        [[-5.42563498e-14]]],


       [[[ 1.76083539e-13]],

        [[-4.36852702e-14]],

        [[ 1.88955376e-12]]],


       [[[-5.42951101e-14]],

        [[ 1.88950807e-12]],

        [[-4.65148211e-14]]]])

In [107]:
def amplitudesToVector(amplitudeTensor):
    amplitudeArray = amplitudeTensor.array

In [108]:
from importlib import reload

In [109]:
import texify
#reload(texify)

In [110]:

texify.texify([Ej, Ek], "CCDEnergyTerms")

In [111]:
t2Tensor.array = trueAmplitudeArray

In [112]:
2 * contractTensorSum(Ej) - contractTensorSum(Ek)

-0.027557222193220297

In [113]:
getDoublesMP2Amplitudes(t2Tensor)

array([[[[-5.93124751e-02]],

        [[-2.94842856e-17]],

        [[-3.08991288e-02]]],


       [[[-2.13644062e-17]],

        [[-3.70716089e-02]],

        [[-2.26545108e-17]]],


       [[[-3.08991288e-02]],

        [[-8.54835499e-18]],

        [[-3.41821777e-02]]]])

In [114]:
t2Tensor.array = getDoublesMP2Amplitudes(t2Tensor)
2 * contractTensorSum(Ej) - contractTensorSum(Ek)

-0.01858327844380032

In [115]:
from pyscf import mp

In [116]:
trueMP2 = mp.MP2(mf)

In [117]:
trueMP2.kernel()

(-0.018583278498233442,
 (array([[[[ 0.00000000e+00, -5.26821666e-18,  1.04083409e-17],
           [ 5.26821666e-18,  0.00000000e+00, -1.66848949e-17],
           [-1.04083409e-17,  1.66848949e-17,  0.00000000e+00]]]]),
  array([[[[-5.93124735e-02,  1.67028935e-18, -3.08991295e-02],
           [ 6.93850601e-18, -3.70716082e-02, -3.90769698e-17],
           [-3.08991295e-02, -2.23920750e-17, -3.41821791e-02]]]]),
  array([[[[ 0.00000000e+00, -5.26821666e-18,  1.04083409e-17],
           [ 5.26821666e-18,  0.00000000e+00, -1.66848949e-17],
           [-1.04083409e-17,  1.66848949e-17,  0.00000000e+00]]]])))